# Knowledge Distillation 🍎🍏🍎

###### Dans ce notebook, on va apprendre à faire du calvados*, et puis on va voir en quoi ça peut etre utile pour le Deep Learning.  

*(ça marche aussi pour le rhum, mais il n'y en a pas en Normandie)

Les alcools sont à peu près tous réalisés de la même manière : on récupère des fruits : des pommes 🍎, dont on fait massérer le jus. Pendant de longues journées (quelques semaines), les levures vont travailler et libérer de précieuses molécules d'éthanol : c'est la FERMENTATION. Après cette longue étape, on a obtenu un alcool dit fermenté : le cidre 🍾 (c'est aussi comme ca qu'on obtient le vin 🍷 ou la bière 🍻 par exemple). Mais, on peut concentrer cet alcool (en un temps relativement faible) en utilisant la DISTILLATION. On obtient alors un alcool distillé (le calvados 🥃) (ou du rhum si vous aviez fait fermenter de la canne à sucre). Mais attention à utiliser la bonne température...

Maintenant, imagine qu'on puisse concentrer rapidement la connaissance qu'un modèle compliqué a mis des heures à apprendre 🍎->🍾, dans un modèle plus simple et plus rapide 🍾->🥃. Tu commences à voir le lien?    

L'idée du Knowledge Distillation c'est qu'un modèle n'a pas forcement besoin des mêmes caractéristiques pour apprendre que pour prédire. Lors de l'apprentissage, un modèle va notament vouloir éviter l'overfitting avec par exemple l'utilisation de dropout. En phase d'apprentissage, on n'a pas forcement besoin d'avoir un modèle léger et rapide. En revanche, pour déployer un modèle et faire des prédictions en temps réel, on peut avoir besoin d'un modèle plus efficace, ce qui implique qu'il soit plus rapide (quitte à être un peu moins précis que le super-modèle original).




Alors commençons par mettre en place les éléments nécessaires (un dataset, et un super modèle qu'on va distiller). On va revenir ensuite plus précisement sur la distillation...

## Libraries, dataset, modèle à distiller : Mise en place de l'alambic...

Dans ce notebook, on va travailler avec la librairie tensorflow. 

`pip install tensorflow`

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt 

Import du dataset MNIST

In [ ]:
(X_train, Y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

print("taille du trainset : "+str(len(X_train)))
print("taille du testset : "+str(len(x_test)))

Petit échauffement : on peut afficher un des éléments du dataset, afin de voir avec quoi on travaille

In [ ]:
%load solutions/print_rd_image.py

On normalise et modifie l'organisation du dataset pour le rendre utilisable par tensorflow

In [ ]:
x_train = X_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = X_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

En se basant sur les meilleures solutions proposées sur kaggle, on peut construire un modèle qui a une précision supérieure à 99,2%. Pour ce genre de problèmes (reconnaissance d'item sur une image), les meilleures solutions sont des réseaux de neurones convolutifs. 


Pour plus d'information sur comment construire un tel modèle :
- https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist 
- https://www.kaggle.com/cdeotte/25-million-images-0-99757-mnist


Le modèle a été entrainé sur 30 périodes, on ne va pas le faire ici, mais si tu veux voir la tête du NN, tu peux load le code ci-dessous. C'est ce modèle qu'on va distiller dans un modèle plus simple et plus petit.

In [ ]:
%load code/MNIST_model.py

In [ ]:
teacher = tf.keras.models.load_model('model/MNIST_model')

Suite et fin de l'échauffement : évaluer le modèle chargé ci-dessus avec le test set. On voit que le modèle réalisé en amont atteint une très bonne accuracy. 

In [ ]:
%load solutions/test_teacher.py

## Revenons à la distillation 

Pour distiller la connaissance d'un modèle (teacher) à un autre (student), la technique est de faire s'entrainer student avec les valeurs de sortie (output layer) du modèle teacher. Autrement dit, la valeur de la fonction objectif (Loss) n'est pas basée sur les Y_train, mais sur les Y_predicted du modèle teacher.
<img src="images/schema1.png"> 
Image en provenance de l'article https://towardsdatascience.com/paper-summary-distilling-the-knowledge-in-a-neural-network-dc8efd9813cc


##### principe de la distillation expliqué :

Mais à quoi cela sert-il? 

Et bien, ce que l'article (https://arxiv.org/abs/1503.02531) met en avant, c'est qu'on va obtenir de meilleure performance en faisant ainsi que en entrainant le modèle student à partir de rien. 

Pourquoi?

Parceque quand on utilise l'output du teacher comme objectif lors de l'entrainement, le modèle student va apprendre plus que seulement "la bonne réponse". 
Imaginons qu'on ait un super modèle (teacher) pour prédire si une image est une pomme verte 🍏, une pomme rouge 🍎, ou une carotte🥕. 

- Pour une instance on a:        y_train[n] = (0,1,0)                       (c'est une pomme rouge 🍎)
- Pour cette même instance on a: y_predict_teacher[n]=(0.1,0.895,0.005)     (c'est une pomme rouge 🍎)

La différence en apprenant avec l'output du teacher c'est que le modèle student ne va pas seulement apprendre que c'est "une pomme rouge". Le modèle student va ausi apprendre que selon le modèle teacher, une pomme rouge ca ressemble plus à une pomme verte qu'à une carotte. 
En faisant ca, student va apprendre non pas à trouver la bonne réponse, mais à réfléchir comme le modèle teacher. Student va apprendre comment le teacher réfléchit. On distille la connaissance du teacher. 


## Impact de la température et du softmax : de l'art de distiller...

Avant même de créer un modèle student, on va s'intéresser à l'impact de la température et de la fonction softmax sur la sortie du super modèle teacher.

Tout d'abord on applique un softmax aux sorties du teacher. Celà permet de normaliser la sortie et d'en faire une distribution de probabilité

\begin{equation*}
q_{i}=\frac{exp(z_{i})}{\sum_{j} exp(z_{j})}
\end{equation*}

Mais en plus de cela on va rajouter une température :
\begin{equation*}
q_{i}=\frac{exp(z_{i} / T)}{\sum_{j} exp(z_{j} / T)}
\end{equation*}

Pour comprendre l'impact de la température, il faut appliquer la fonction ci-dessus aux predictions du modèle teacher. A toi de jouer...

`tf.nn.softmax()` (fonction utile...)

In [ ]:
%load solutions/temperature.py

On doit voir deux choses :
- le softmax normalise bien les sorties
- une augmentation (T>1) de la température compresse les valeurs de sortie (la pomme verte 🍏 ressemble vraiment à la pomme rouge 🍎), une diminution (T<1) de la température dilate les valeurs de sortie (Pas de place au doute, cette image est vraiment une pomme rouge🍎🍎)


Ainsi, il faudra déterminer empiriquement quelle va être la bonne température à utiliser lors de la distillation, afin de transmettre la connaissance (T suffisement grand), sans non plus noyer la bonne réponse (T suffisement petit)(sinon : (0.325🍏,0.35🍎,0.325🥕) pas sûr que ce soit une pomme rouge, ...)

## Construction du modèle student

Voici le modèle student qu'on propose de mettre en place. Il est beaucoup moins lourd que le modèle teacher, mais est quand même très approprié pour répondre à ce genre de problème

In [ ]:
def stud_mod():
    model = keras.Sequential(
        [
            keras.Input(shape=(28, 28, 1)),
            layers.Conv2D(16, 5, strides=1, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.MaxPooling2D(pool_size=2, padding="same"),
            layers.Flatten(),
            layers.Dense(10),
        ]
        )
    return model


## Construction de la classe distiller

C'est là que tout se joue. Tout est fait pour toi, on va juste essayer de faire l'implémentation du calcul du loss. Trois aides / indices:
- le calcul de la distillation_loss nécessite le softmax et la température vus juste avant. 
- dans le calcul du loss total, on va pondérer par un paramère alpha pour garder une influence du student loss (entrainement classique sans distillation)
- les attributs self.distillation_loss_fn et self.student_loss_fn sont des méthodes de calcul du loss, ils sont utilisés ainsi : self.loss_fn(y_true,y_predicted)

In [ ]:
## A compléter 

class Distiller(keras.Model):
    #on crée une classe inspirée de la classe keras.Model : on ne va pas recoder tout ce qui existe déjà. 
    
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=1,
    ):
       
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature
        # la compilation va changer en quelques points : on a besoin de rajouter un paramètre de température, et un paramètre alpha 

    def train_step(self, data):
        # c'est là qu'on a le plus de travail à faire : on overwrite la fonction train_step de keras.Model

        x, y = data
        teacher_predictions = self.teacher(x, training=False)
        with tf.GradientTape() as tape:
            student_predictions = self.student(x, training=True)

            # calcul du loss : à toi de jouer
            student_loss = ###???###
            distillation_loss = ###???###
            loss = ###???###
            # fin du jeu
            
 
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.compiled_metrics.update_state(y, student_predictions)
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        x, y = data
        y_prediction = self.student(x, training=False)
        student_loss = self.student_loss_fn(y, y_prediction)
        self.compiled_metrics.update_state(y, y_prediction)
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [ ]:
## SOLUTION

# calcul du loss : à toi de jouer
student_loss = self.student_loss_fn(y, student_predictions)
distillation_loss = self.distillation_loss_fn(
    tf.nn.softmax(teacher_predictions / self.temperature),
    tf.nn.softmax(student_predictions / self.temperature),
)
loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss
# fin du jeu

## Distillons ("du calvados, enfin!!") 🥃

Le but est d'implémenter une distillation complète. Le temps restant peut etre consacré à regarder l'impact de la température et de alpha sur l'apprentissage par student. 

NB: pour les arguments de compile dans le distiller, on propose d'utiliser les calculs de loss suivants:
`student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True)`
 `distillation_loss_fn=keras.losses.KLDivergence()`

In [ ]:
%load solutions/distillation.py

Pour te faire gagner du temps, plusieurs distillation ont été lancées avec un paramètre T différent (avec alpha=0.1). Voilà ce que ça donne :
<img src="images/aT.jpg"> 

Pour T=1, plusieurs distillations sont réalisées afin de voir l'inflence du paramètre alpha:
<img src="images/alpha_n_mean.jpg"> 

Pour alpha=1, c'est comme-ci on n'avait pas de distillation, mais seulement un apprentissage classique (cf calcul du loss dans la class distiller)

## Conclusion

Les résultats sont décevants par rapport au papier et aux exemples trouvés. Néanmoins, le dataset utilisé ici est assez simple, ce qui explique pourquoi le NN student n'a pas de mal à l'apprendre même sans distillation (d'autant que student est aussi un réseau convolutif, qui est particulièrement adapté à ce genre de dataset). Là où la distillation devient très intéressante, c'est sur des modèles plus compliqués, voir sur des ensembles de modèle, avec plus de classes en sortie.

## Ce qu'il faut retenir

Le knowledge distillation a été vu ici sur un exmple simple. Son utilité est surtout valable dans le cas de modèle plus complexe, sur des datasets avec plus de classes en sorties. 

- Le knoledge distillation, c'est le fait d'entrainer un modèle (student 🥃) a partir des outputs d'un autre modèle (teacher 🍾🍎)
- La température permet de compresser les résultats du modèle teacher, c'est un facteur important de la distillation, et il doit etre trouvé empiriquement
- C'est une technique à vraiment garder à l'esprit quand on a des modèles trop lourd à manipuler et mais qu'on ne veut pas perdre l'intégralité de la connaissance qu'ils ont emmagasiné
- C'est une technique relativement bien documentée (l'article initial date de 2015)


Des sources et liens intéressants:
- https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist 
- https://www.kaggle.com/cdeotte/25-million-images-0-99757-mnist
- https://arxiv.org/abs/1503.02531
- https://towardsdatascience.com/paper-summary-distilling-the-knowledge-in-a-neural-network-dc8efd9813cc
- https://www.youtube.com/watch?v=k63qGsH1jLo&t=2s
- https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/vision/ipynb/knowledge_distillation.ipynb#scrollTo=4qtExPtWgiV1
- https://keras.io/examples/vision/knowledge_distillation/
- https://github.com/karanchahal/distiller